### PROJETO 1 REDES SOCIAIS

**GRUPO D:** 
- ESTHER DAGIR
- FELIPE LEMOS
- JV PAZOTTI

**DESCRIÇÃO DATASET CORD19**

Artigos acadêmicos relacionados ao COVID-19 e suas variantes que integram o dataset CORD-19. Coleção mais extensa (+134.000 artigos) de pesquisa sobre coronavírus legível por máquina disponível para mineração de dados até o momento, solicitada pelo governo americano em conjunto com entidades de pesquisa sobre o vírus e suas variantes. 

### Importando as Bibliotecas


In [ ]:
import pandas as pd
import seaborn as sns
import netpixi
from netpixi.integration.gt import *
from regression.integration.gt import *

### Dataset

In [ ]:
data = pd.read_csv("metadata.csv")

In [ ]:
unique_journals = data['journal'].unique().tolist()
len(unique_journals)

In [ ]:
unique_df = data.drop_duplicates(subset=['journal'])

In [ ]:
data=unique_df
unique_df.shape

In [ ]:
data.info()

In [ ]:
data=data.copy().drop(columns=['pdf_json_files','pmc_json_files','arxiv_id','pmcid','mag_id','s2_id'])

In [ ]:
data['authors'] = data['authors'].astype(str)

### Construindo a rede de coautoria

In [ ]:
g = Graph(directed=False) # não dirigido

In [ ]:
collabs = {}

# percorre todos os autores no dataset
for authors in data['authors']:
    # divide os autores em uma lista
    authors_list = authors.split('; ')
    # percorre todos os pares de autores na lista
    for i in range(len(authors_list)):
        for j in range(i+1, len(authors_list)):
            # ordena os nomes dos autores em ordem alfabética para evitar duplicatas
            pair = tuple(sorted([authors_list[i], authors_list[j]]))
            # incrementa o contador de co-autorias para esse par de autores
            collabs[pair] = collabs.get(pair, 0) + 1    
            
co_authors = [] #todos os influentes

# seleciona os autores que fizeram pelo menos 2 co-autorias (threshold)
for pair, count in collabs.items():
    if count >= 3: 
        # adiciona os autores da co-autoria à lista de autores colaboradores influentes
        co_authors.extend(pair)
        
unique_authors = set(co_authors)

In [ ]:
import numpy as np
np.mean(list(collabs.values()))

In [ ]:
for author in unique_authors:
    g.add_vertex(author) 
    
for pair, count in collabs.items():
    try:
        if count >= 3 and pair[0] in unique_authors and pair[1] in unique_authors:
            if pair[0]!= pair[1]:
                g.add_edge(pair[0], pair[1])
    except ValueError as e:
        print("Erro ao adicionar a aresta {}: {}".format(pair, e))


In [ ]:
cord19 = gt_data(g)

In [ ]:
n = g.num_vertices()
m =  g.num_edges()
max_edges = n * (n - 1) if g.is_directed() else n * (n - 1) // 2
d = m / max_edges

print('Número de vértices de g:', n)
print('Número de arestas de g:', m)
print('Máximo possível de arestas de g:', max_edges)
print('Densidade de g:', d)

In [ ]:
for v in g.all_vertices():
    if v.total_degree()==1:
        try:
            g.remove_vertex(v['id'])
        except ValueError as e:
            print("Erro ao retirar o vertice {}: {}".format(v, e))

In [ ]:
in_degrees = [] 
for v in g.all_vertices():
    in_degrees.append(v.in_degree())

out_degrees = []
for v in g.all_vertices():
    out_degrees.append(v.out_degree())

degrees = []
for v in g.all_vertices():
    degrees.append(v.total_degree())
    
cord19['in_degree'] = in_degrees
cord19['out_degree'] = out_degrees
cord19['degree'] = degrees
cord19

In [ ]:
cord19['degree'].value_counts()

In [ ]:
cord19.to_csv('cord19.csv')

In [ ]:
sns.histplot(cord19['degree'],bins=200)
cord19['degree'].describe()

In [ ]:
gt_save(g, './cord19_graph.net.gz')
g = gt_load('./cord19_graph.net.gz')
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
r=netpixi.render('./cord19_graph.net.gz')

In [ ]:
len(co_authors)
from graph_tool import spectral
import cpnet

In [ ]:
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, './cord19_graph.net.gz')
r = netpixi.render('./cord19_graph.net.gz', infinite=True)

In [ ]:
r.vertex_default(color=0x09DB8F ,size=5)
r.edge_default(color=0xFFFFFF, width=0.5)

In [ ]:
matrix = spectral.adjacency(g)

# Além de cpnet.Rombach, há outras
# opções de algoritmos contínuos:
# cpnet.MINRES
# cpnet.Rossa

algorithm = cpnet.Rombach()
algorithm.detect(matrix)
c = algorithm.get_coreness()

# Depois do código acima, c é um dicionário
# em que as chaves são índices de vértices e
# valores são os atribuídos pelo algoritmo.

g.add_vp('coreness')
for i, coreness in c.items():
    v = g.get_vertex_by_index(i)
    v['coreness'] = float(coreness)

# Vamos usar coreness como referência para
# o tamanho dos vértices. Estabelecemos 10
# como o menor tamanho possível e 40 como
# o maior tamanho possível. Como o coreness
# está entre 0 e 1, a conta é bem simples.

for v in g.all_vertices():
    r.vertex(v['id'], size=(10 + 40 * v['coreness']))

# Quando a execução desta célula terminar,
# veja como ficou a visualização acima.